<a href="https://colab.research.google.com/github/sajidcsecu/radioGenomic/blob/main/3DUnetinGPU_(Nifti_MONAI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the Code for the Segmentation on Rider Dataset (LUNG1). The Code is worked on the 3D volume over GPU. The balanced sampler, preprocessed data (uniform volume spacing and clipping [-1000, 700]) and the strong augmentation is used in the code...

# (1) Import Required Libraries

In [ ]:
!pip install SimpleITK
!pip install pydicom===2.4.3
!pip install pydicom-seg
!pip install numpy==1.23.5
!pip install monai
!pip install torch==1.13.1
!pip install nibabel>=5.0.0

  Using cached monai-1.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached monai-1.5.0-py3-none-any.whl (2.7 MB)
Using 

  Using cached torch-1.13.1-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
Using cached torch-1.13.1-cp311-cp311-manylinux1_x86_64.whl (887.4 MB)
   ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 139.1/317.1 MB 227.7 MB/s eta 0:00:01^C



# (2) Import required Libraries

In [1]:
import os
from pathlib import Path
import csv
import time
import random
import shutil
from glob import glob
from typing import List
import torch
import numpy as np
import torch.nn as nn
import torch.cuda.amp as amp
from torch.optim import lr_scheduler
from monai.inferers import sliding_window_inference
from monai.transforms import AsDiscrete
from monai.networks.nets import UNet
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd,
    Orientationd, ScaleIntensityRanged, CropForegroundd, Resized,
    RandFlipd, RandFlipd, RandAffined, RandGaussianNoised, RandScaleIntensityd,EnsureTyped, ToTensord
)
from monai.data import Dataset, DataLoader, CacheDataset, pad_list_data_collate
from monai.networks.layers import Norm
import nibabel as nib
from sklearn.metrics import jaccard_score, f1_score, recall_score, precision_score, accuracy_score
import matplotlib.pyplot as plt
import multiprocessing as mp
from monai.transforms import EnsureTyped
from monai.transforms import SaveImaged
from monai.utils import set_determinism

# (3) Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## (4). Loss Function

In [ ]:
class DiceBCELoss3D(nn.Module):
    def __init__(self, smooth=1e-6, epsilon=1e-8):
        super().__init__()
        self.smooth = smooth
        self.epsilon = epsilon
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, preds, targets):
        preds = preds.flatten()
        targets = targets.flatten()
        preds_sigmoid = torch.sigmoid(preds)
        intersection = (preds_sigmoid * targets).sum()
        dice_loss = 1 - (2. * intersection + self.smooth) / (
            preds_sigmoid.sum() + targets.sum() + self.smooth + self.epsilon)
        bce_loss = self.bce(preds, targets)
        return dice_loss + bce_loss

# (5). Test

In [ ]:
class UnetTest:
    def __init__(self, test_result_path: str, metrics_csv: str, device: torch.device):
        self.test_result_path = test_result_path
        self.metrics_csv = metrics_csv
        self.device = device

        os.makedirs(self.test_result_path, exist_ok=True)
        self._init_metrics_csv()

    def _init_metrics_csv(self):
        if not os.path.exists(self.metrics_csv):
            with open(self.metrics_csv, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["SampleID", "Jaccard", "F1", "Recall", "Precision", "Accuracy", "Time"])

    def calculate_metrics(self, y_true: np.ndarray, y_pred: np.ndarray):
        y_true = y_true.astype(bool).flatten()
        y_pred = y_pred.astype(bool).flatten()

        return [
            jaccard_score(y_true, y_pred, zero_division=0),
            f1_score(y_true, y_pred, zero_division=0),
            recall_score(y_true, y_pred, zero_division=0),
            precision_score(y_true, y_pred, zero_division=0),
            accuracy_score(y_true, y_pred)
        ]

    def save_result_slices(self, image: np.ndarray, pred_mask: np.ndarray, true_mask: np.ndarray, sample_id: str):
        sample_dir = os.path.join(self.test_result_path, sample_id)
        os.makedirs(sample_dir, exist_ok=True)

        for i in range(image.shape[0]):
            try:
                fig, ax = plt.subplots(1, 3, figsize=(12, 4))
                ax[0].imshow(image[i], cmap='gray')
                ax[0].set_title('Image')

                ax[1].imshow(true_mask[i], cmap='gray')
                ax[1].set_title('Ground Truth')

                ax[2].imshow(pred_mask[i], cmap='gray')
                ax[2].set_title('Prediction')

                for a in ax:
                    a.axis('off')
                plt.tight_layout()
                plt.savefig(os.path.join(sample_dir, f'slice_{i:03d}.png'))
                plt.close()
            except Exception as e:
                print(f"⚠️ Could not save slice {i} for {sample_id}: {e}")

    def append_metrics_to_csv(self, sample_id: str, metrics: list, elapsed_time: float):
        with open(self.metrics_csv, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([sample_id] + [f"{m:.4f}" for m in metrics] + [f"{elapsed_time:.4f}"])

    def test(self, model: nn.Module, test_loader: DataLoader):
        model.eval()
        total_metrics = np.zeros(5)
        total_times = []

        roi_size = (96, 96, 96)
        sw_batch_size = 1

        with torch.no_grad():
            for batch_idx, batch in enumerate(test_loader):
                image, label = batch["vol"].to(self.device), batch["seg"].to(self.device)
                start_time = time.time()

                pred = sliding_window_inference(
                    inputs=image,
                    roi_size=roi_size,
                    sw_batch_size=sw_batch_size,
                    predictor=model
                )
                pred = torch.sigmoid(pred) > 0.5  # Binary thresholding

                elapsed = time.time() - start_time
                total_times.append(elapsed)

                # Convert to NumPy
                image_np = image[0, 0].cpu().numpy()
                label_np = label[0, 0].cpu().numpy()
                pred_np = pred[0, 0].cpu().numpy()

                # Metrics
                metrics = self.calculate_metrics(label_np, pred_np)
                total_metrics += np.array(metrics)

                # Identify sample name
                sample_id = os.path.basename(batch["vol_meta_dict"]["filename_or_obj"][0]).replace(".nii.gz", "")
                self.save_result_slices(image_np, pred_np, label_np, sample_id)
                self.append_metrics_to_csv(sample_id, metrics, elapsed)

        # Print summary
        num_samples = len(test_loader)
        print("\n📊 Average Test Metrics:")
        print(f"Jaccard:  {total_metrics[0]/num_samples:.4f}")
        print(f"F1:       {total_metrics[1]/num_samples:.4f}")
        print(f"Recall:   {total_metrics[2]/num_samples:.4f}")
        print(f"Precision:{total_metrics[3]/num_samples:.4f}")
        print(f"Accuracy: {total_metrics[4]/num_samples:.4f}")
        print(f"⚡ FPS:    {1 / np.mean(total_times):.2f}")

# (6). Training

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=True, min_delta=0, path='checkpoint.pt',
                 start_val_loss_min=None, start_patience_counter=0):
        self.patience = patience
        self.verbose = verbose
        self.min_delta = min_delta
        self.path = path
        self.val_loss_min = start_val_loss_min if start_val_loss_min is not None else np.inf
        self.counter = start_patience_counter
        self.early_stop = False

    def __call__(self, val_loss, model, epoch=None, optimizer=None):
        improved = False
        if val_loss < self.val_loss_min - self.min_delta:
            self.val_loss_min = val_loss
            self.counter = 0
            improved = True
            if self.verbose:
                print(f"✅ Validation loss improved. Saving model...")
        else:
            self.counter += 1
            if self.verbose:
                print(f"⏳ EarlyStopping counter: {self.counter} out of {self.patience}")

        # ✅ Always save full checkpoint (model + optimizer + val_loss + patience_counter)
        self.save_checkpoint(model, epoch, optimizer)

        if self.counter >= self.patience:
            self.early_stop = True

        return self.early_stop

    def save_checkpoint(self, model, epoch=None, optimizer=None):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict() if optimizer else None,
            'val_loss': self.val_loss_min,
            'patience_counter': self.counter
        }
        torch.save(checkpoint, self.path)




class UnetTrain:
    def __init__(self, model_file, loss_result_path, lr, num_epochs, device):
        self.model_file = model_file
        self.loss_result_path = loss_result_path
        self.lr = lr
        self.num_epochs = num_epochs
        self.device = device
        self.seeding(42)

    def seeding(self, seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    def epoch_time(self, start_time, end_time):
        elapsed = end_time - start_time
        return int(elapsed / 60), int(elapsed % 60)

    def train_one_epoch(self, model, loader, optimizer, loss_fn):
        model.train()
        epoch_loss = 0
        scaler = torch.amp.GradScaler()  # no device_type here

        device_type = 'cuda' if self.device.type == 'cuda' else 'cpu'

        for x in loader:
            inputs, labels = x["vol"].to(self.device), x["seg"].to(self.device)
            optimizer.zero_grad()
            with torch.amp.autocast(device_type=device_type):
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            epoch_loss += loss.item()

        return epoch_loss / len(loader)

    def evaluate(self, model, loader, loss_fn):
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for x in loader:
                inputs, labels = x["vol"].to(self.device), x["seg"].to(self.device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                epoch_loss += loss.item()
        return epoch_loss / len(loader)

    def execute(self, train_loader, valid_loader):
        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=1,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH
        ).to(self.device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=self.lr, weight_decay=1e-5)
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
        loss_fn = DiceBCELoss3D()

        # Initialize state
        start_epoch = 1
        start_val_loss_min = None
        start_patience_counter = 0
        history = {"train_loss": [], "valid_loss": []}

        # 📦 Restore from model checkpoint if exists
        if os.path.exists(self.model_file):
            checkpoint = torch.load(self.model_file, map_location=self.device)
            model.load_state_dict(checkpoint['model_state_dict'])
            if checkpoint.get('optimizer_state_dict'):
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint.get('epoch', 1) + 1
            start_val_loss_min = checkpoint.get('val_loss', None)
            start_patience_counter = checkpoint.get('patience_counter', 0)

        # 📊 Restore training history from loss CSV
        if os.path.exists(self.loss_result_path):
            with open(self.loss_result_path, 'r') as f:
                reader = csv.reader(f)
                next(reader)
                rows = list(reader)
                if rows:
                    last_epoch = int(rows[-1][0])
                    start_epoch = last_epoch + 1
                    history['train_loss'] = [float(r[1]) for r in rows]
                    history['valid_loss'] = [float(r[2]) for r in rows]
                    if start_val_loss_min is None:
                        start_val_loss_min = min(history['valid_loss'])

            # 💾 Make a backup copy
            backup_path = self.loss_result_path.replace(".csv", "_backup.csv")
            shutil.copy(self.loss_result_path, backup_path)

        # 🛑 Initialize EarlyStopping
        early_stopping = EarlyStopping(
            patience=10,
            min_delta=0.0005,
            path=self.model_file,
            start_val_loss_min=start_val_loss_min,
            start_patience_counter=start_patience_counter
        )

        # 📁 If loss file not present, create CSV header
        if not os.path.exists(self.loss_result_path):
            with open(self.loss_result_path, "w", newline="") as f:
                csv.writer(f).writerow(["Epoch", "Train Loss", "Valid Loss"])

        # 🚂 Training Loop
        for epoch in range(start_epoch, self.num_epochs + 1):
            start_time = time.time()

            train_loss = self.train_one_epoch(model, train_loader, optimizer, loss_fn)
            valid_loss = self.evaluate(model, valid_loader, loss_fn)
            scheduler.step()

            epoch_mins, epoch_secs = self.epoch_time(start_time, time.time())
            print(f"Epoch {epoch:03d} | Time: {epoch_mins}m {epoch_secs}s | "
                  f"Train: {train_loss:.6f} | Val: {valid_loss:.6f}")

            history['train_loss'].append(train_loss)
            history['valid_loss'].append(valid_loss)

            with open(self.loss_result_path, "a", newline="") as f:
                csv.writer(f).writerow([epoch, train_loss, valid_loss])

            # 🛑 Early stopping check
            if early_stopping(valid_loss, model, epoch, optimizer):
                print("🛑 Early stopping triggered.")
                break

            torch.cuda.empty_cache()


# (7). Pipeline

In [ ]:
class UnetPipeline:
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.setup_paths()
        print("📦 Loading datasets...")
        self.train_loader, self.valid_loader, self.test_loader = self.prepare_loaders()

    def setup_paths(self):
        os.chdir(self.config['target_dir'])
        self.output_dir = os.path.join(".", "results", self.config['output_folder_name'])
        os.makedirs(self.output_dir, exist_ok=True)

        self.loss_result_file = os.path.join(self.output_dir, "train_and_valid_loss_results.csv")
        self.model_file = os.path.join(self.output_dir, "model.pth")
        self.test_metrics_file = os.path.join(self.output_dir, "test_metrics.csv")
        self.test_result_path = os.path.join(self.output_dir, "test_outputs")
        os.makedirs(self.test_result_path, exist_ok=True)

        self.dataset_dir = os.path.join("./datasets", f"Datasets_{self.config['transformation']}")

    def prepare_loaders(self):
        from glob import glob
        from monai.transforms import (
            Compose, LoadImaged, EnsureChannelFirstD, Spacingd, Orientationd,
            ScaleIntensityRanged, CropForegroundd, Resized, ToTensord,
            RandFlipd, RandAffined, RandGaussianNoised, RandScaleIntensityd
        )
        from monai.data import Dataset, DataLoader

        pixdim = (1, 1, 1)
        a_min, a_max = -1000, 700
        spatial_size = (96, 96, 96)

        def get_files(split):
            ct = sorted(glob(os.path.join(self.dataset_dir, split, "ct", "*.nii.gz")))
            seg = sorted(glob(os.path.join(self.dataset_dir, split, "segment", "*.nii.gz")))
            return [{"vol": c, "seg": s} for c, s in zip(ct, seg)]

        # Training transforms with augmentation
        train_transforms = Compose([
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstD(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),

            # ➕ Augmentations
            RandFlipd(keys=["vol", "seg"], prob=0.5, spatial_axis=0),
            RandFlipd(keys=["vol", "seg"], prob=0.5, spatial_axis=1),
            RandAffined(
                keys=["vol", "seg"],
                prob=0.3,
                rotate_range=(0.1, 0.1, 0.1),
                scale_range=(0.1, 0.1, 0.1),
                mode=("bilinear", "nearest")
            ),
            RandGaussianNoised(keys=["vol"], prob=0.2, mean=0.0, std=0.1),
            RandScaleIntensityd(keys=["vol"], factors=0.1, prob=0.5),

            ToTensord(keys=["vol", "seg"])
        ])

        # Validation/test transforms without augmentation
        base_transforms = Compose([
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstD(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),
            ToTensord(keys=["vol", "seg"])
        ])

        train_loader = DataLoader(Dataset(get_files("train"), train_transforms), batch_size=self.config['batch_size'], shuffle=True)
        valid_loader = DataLoader(Dataset(get_files("valid"), base_transforms), batch_size=self.config['batch_size'])
        test_loader = DataLoader(Dataset(get_files("test"), base_transforms), batch_size=1)

        return train_loader, valid_loader, test_loader

    def train(self):
        trainer = UnetTrain(
            model_file=self.model_file,
            loss_result_path=self.loss_result_file,
            lr=self.config['learning_rate'],
            num_epochs=self.config['num_epochs'],
            device=self.device
        )
        trainer.execute(self.train_loader, self.valid_loader)

    def test(self):
        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=1,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH
        ).to(self.device)
        checkpoint = torch.load(self.model_file, map_location=self.device)
        model.load_state_dict(checkpoint['model_state_dict'])

        tester = UnetTest(self.test_result_path, self.test_metrics_file, self.device)
        tester.test(model, self.test_loader)

    def run(self):
        self.train()
        self.test()


def main():
    config = {
        'target_dir': "/content/drive/MyDrive/PhDwork/Segmentation",
        'output_folder_name': "Results_MONAI_Augmented",
        'transformation': "OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train",
        'batch_size': 2,
        'num_epochs': 100,
        'learning_rate': 1e-4,
    }
    pipeline = UnetPipeline(config)
    pipeline.run()


if __name__ == "__main__":
    mp.set_start_method('spawn')
    main()


📦 Loading datasets...
Epoch 071 | Time: 71m 35s | Train: 0.674845 | Val: 0.805939
⏳ EarlyStopping counter: 2 out of 10
Epoch 072 | Time: 68m 22s | Train: 0.711546 | Val: 0.810964
⏳ EarlyStopping counter: 3 out of 10
Epoch 073 | Time: 68m 42s | Train: 0.706732 | Val: 0.791821
⏳ EarlyStopping counter: 4 out of 10
Epoch 074 | Time: 68m 55s | Train: 0.681764 | Val: 0.789887
⏳ EarlyStopping counter: 5 out of 10
Epoch 075 | Time: 69m 1s | Train: 0.665208 | Val: 0.781649
✅ Validation loss improved. Saving model...
Epoch 076 | Time: 69m 7s | Train: 0.641370 | Val: 0.784236
⏳ EarlyStopping counter: 1 out of 10
Epoch 077 | Time: 69m 18s | Train: 0.660566 | Val: 0.789251
⏳ EarlyStopping counter: 2 out of 10
Epoch 078 | Time: 69m 23s | Train: 0.639254 | Val: 0.780697
✅ Validation loss improved. Saving model...
Epoch 079 | Time: 69m 26s | Train: 0.631953 | Val: 0.774602
✅ Validation loss improved. Saving model...
Epoch 080 | Time: 69m 35s | Train: 0.628322 | Val: 0.770767
✅ Validation loss improved

#(8) Mask Generation

In [4]:
import os
import torch
import nibabel as nib
import numpy as np
from pathlib import Path
from monai.networks.nets import UNet
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    EnsureTyped,
    ToTensord,
)
from monai.data import Dataset, DataLoader, list_data_collate
from monai.inferers import sliding_window_inference
from monai.utils import set_determinism
from monai.networks.layers import Norm


class UNetInferencePipeline:
    def __init__(self, model_path, input_folder, output_folder, device="cuda:0"):
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")
        self.model_path = model_path
        self.input_folder = input_folder
        self.output_folder = output_folder
        os.makedirs(output_folder, exist_ok=True)

        set_determinism(seed=42)

        self.model = self._load_model()
        self.transform = self._prepare_transforms()
        self.dataloader = self._prepare_dataloader()

    def _load_model(self):
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model file not found at: {self.model_path}")

        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=1,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH
        ).to(self.device)

        state_dict = torch.load(self.model_path, map_location=self.device)
        if 'model_state_dict' in state_dict:
            model.load_state_dict(state_dict['model_state_dict'])
        else:
            model.load_state_dict(state_dict)

        model.eval()
        print(f"✅ Model loaded successfully from {self.model_path}")
        return model

    def _prepare_transforms(self):
        return Compose([
            LoadImaged(keys=["vol"]),
            EnsureChannelFirstd(keys=["vol"]),
            Spacingd(keys=["vol"], pixdim=(1.0, 1.0, 1.0), mode="bilinear"),
            Orientationd(keys=["vol"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-1000, a_max=700, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=["vol"], source_key="vol"),
            EnsureTyped(keys=["vol"], track_meta=True),
            ToTensord(keys=["vol"]),
        ])

    def _prepare_dataloader(self):
        nifti_files = sorted(Path(self.input_folder).glob("*.nii.gz"))
        if not nifti_files:
            raise FileNotFoundError(f"No NIfTI files found in input folder: {self.input_folder}")

        data_dicts = [{"vol": str(f)} for f in nifti_files]
        print(f"🔍 Found {len(data_dicts)} NIfTI files for inference.")
        return DataLoader(Dataset(data=data_dicts, transform=self.transform),
                          batch_size=1, num_workers=0, collate_fn=list_data_collate)

    def infer(self, roi_size=(96, 96, 96), sw_batch_size=1, overlap=0.5):
        self.model.eval()
        with torch.no_grad():
            for i, batch in enumerate(self.dataloader):
                try:
                    meta = batch["vol"].meta if hasattr(batch["vol"], "meta") else {}
                    input_path = meta.get("filename_or_obj", [f"case_{i}"])[0]
                    input_filename = Path(input_path).stem

                    images = batch["vol"].to(self.device)
                    print(f"🔍 Inference on [{i+1}] {input_filename} | shape = {images.shape}")

                    # Sliding window inference
                    preds = sliding_window_inference(
                        images, roi_size=roi_size,
                        sw_batch_size=sw_batch_size,
                        predictor=self.model,
                        overlap=overlap
                    )

                    # Apply sigmoid and threshold
                    probs = torch.sigmoid(preds)
                    pred_label = (probs > 0.5).float()

                    # Save output
                    self._save_prediction(
                        pred=pred_label,
                        meta=meta,
                        input_filename=input_filename
                    )

                except Exception as e:
                    print(f"❌ Error during inference on batch {i}: {e}")


    def _save_prediction(self, pred, meta, input_filename):
        # Remove batch and channel dimensions: [1, 1, D, H, W] → [D, H, W]
        pred_np = pred.squeeze().cpu().numpy()

        # --- Robust affine extraction ---
        affine = meta.get("affine")
        if isinstance(affine, list):
            affine = affine[0]
        affine = np.asarray(affine)
        if affine.shape != (4, 4):
            print(f"⚠️ Invalid affine shape: {affine.shape}, using identity affine.")
            affine = np.eye(4)

        # Save with nibabel
        out_path = Path(self.output_folder) / f"{input_filename}_seg.nii.gz"
        nib_img = nib.Nifti1Image(pred_np.astype(np.uint8), affine)
        nib.save(nib_img, str(out_path))
        print(f"✅ Saved: {out_path}")

if __name__ == "__main__":
    ROOT_DIR = "/content/drive/MyDrive/PhDwork/Segmentation"
    MODEL_PATH = os.path.join(ROOT_DIR, "results", "Results_MONAI_Augmented", "model.pth")
    INPUT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "Lung3", "ct")
    OUTPUT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "Lung3", "segment")

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    os.chdir(ROOT_DIR)

    try:
        pipeline = UNetInferencePipeline(MODEL_PATH, INPUT_FOLDER, OUTPUT_FOLDER)
        pipeline.infer()
        print("🎉 Inference completed successfully for all patients!")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


✅ Model loaded successfully from /content/drive/MyDrive/PhDwork/Segmentation/results/Results_MONAI_Augmented/model.pth
🔍 Found 89 NIfTI files for inference.
🔍 Inference on [1] LUNG3-01.nii | shape = torch.Size([1, 1, 58, 499, 2553])
⚠️ Invalid affine shape: (1, 4, 4), using identity affine.
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/Lung3/segment/LUNG3-01.nii_seg.nii.gz
🔍 Inference on [2] LUNG3-02.nii | shape = torch.Size([1, 1, 40, 360, 2556])
⚠️ Invalid affine shape: (1, 4, 4), using identity affine.
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/Lung3/segment/LUNG3-02.nii_seg.nii.gz
🔍 Inference on [3] LUNG3-03.nii | shape = torch.Size([1, 1, 44, 364, 2556])
⚠️ Invalid affine shape: (1, 4, 4), using identity affine.
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/Lung3

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List
import os
import csv


class LossPlotter:
    def __init__(self, csv_path: str):
        self.csv_path = Path(csv_path)
        self.data = self._load_data()

    def _load_data(self):
        if not self.csv_path.exists():
            raise FileNotFoundError(f"CSV file not found: {self.csv_path}")
        df = pd.read_csv(self.csv_path, index_col=0)  # Read row labels as index
        return df  # Make rows into columns

    def plot(self, title: str = "Training and Validation Loss", save_path= None):
        plt.figure(figsize=(8, 5))
        plt.plot(self.data.index, self.data['Train Loss'], label='Train Loss', color='blue')
        plt.plot(self.data.index, self.data['Valid Loss'], label='Valid Loss', color='orange')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(title)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        if save_path:
            save_path = Path(save_path)
            save_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, format='pdf')
            print(f"[INFO] Loss plot saved to {save_path}")
        else:
            plt.show()

        plt.close()

if __name__ == "__main__":
    target_dir = "/content/drive/MyDrive/PhDwork/Segmentation"
    os.chdir(target_dir)
    loss_result_file = os.path.join(".","results",f"Results_PreProcessedCT_Fifty_Fifty_DiceLoss_And_Strong_Augmentation","train_and_valid_loss_results.csv")
    plotter = LossPlotter(loss_result_file)
    plotter.plot()


In [ ]:
import h5py
os.chdir("/content/drive/MyDrive/PhDwork/Segmentation")
print(f"📁 Current Directory: {os.getcwd()}")
with h5py.File('./datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/train_dataset.hdf5', 'r') as f:
    print(list(f.keys()))